아래의 2016-10-20의 뉴스로부터 명사를 추출하여 미리 토크나이즈를 해둔 tokenized_corpus_fname으로부터 term frequency matrix인 x와 index2word를 만들어 뒀습니다. 

In [1]:
import config
from navernews_10days import get_bow

x, _idx_to_vocab, _vocab_to_idx = get_bow(tokenize='noun', date='2016-10-20')

soynlp=0.0.492
added lovit_textmining_dataset


In [2]:
import soykeyword
print(soykeyword.__version__)

0.0.14


## Lasso regression for keyword extraction

Lasso regression 을 이용하여 키워드를 찾는 과정을 패키징하여 soykeyword 에 올려두었습니다. 설치는 pip install 이 가능합니다. 

    pip install soykeyword

LassoKeywordExtractor는 Lasso Logistic regression을 이용하여 positive documents와 negative (or refeerence) documents를 잘 구분하는 단어들을 선택합니다. 이를 위해 scikit-learn의 Logistic Regression을 이용하였습니다.

sparse matrix 형식의 x를 LassoKeywordExtractor의 train()의 argument로 입력받으며, minimum term frequency와 minimum document frequency로 키워드 후보를 제한하기 쉽도록 LassoKeywordExtractor 생성자의 arguments를 넣어뒀습니다. Logistic regression이 이용하는 regularization cost coefficient C는 costs라는 list of number로 입력됩니다. 

LassoKeywordExtractor는 텍스트 형식의 데이터는 지원하지 않습니다. Sparse matrix 형식에만 지원하며, train 단계에서 index2word를 입력하지 않으면 <키워드 아이디, 빈도수, Logistic regression coefficient> 형식의 namedtuple인 KeywordScore로 출력됩니다. 

min_num_of_keywords L1 regularization cost를 조금씩 낮춰가면서 선택되는 최소한의 키워드의 개수가 min_num_of_keywords 이상이 되도록 하는 최초의 keywords를 return하도록 하는 parameter입니다. 

In [3]:
from soykeyword.lasso import LassoKeywordExtractor

lassobased_extractor = LassoKeywordExtractor(
    costs=[500, 200, 100, 50, 10, 5, 1, 0.1],
    min_tf=20, 
    min_df=10
)

lassobased_extractor.train(x)
keywords = lassobased_extractor.extract_from_word(
    5537, min_num_of_keywords=30)
keywords[:10]

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


28 keywords extracted from 0.100 cost
63 keywords extracted from 1.000 cost


[KeywordScore(word=2308, frequency=86, coefficient=3.8160088460462993),
 KeywordScore(word=4701, frequency=40, coefficient=3.25778936801015),
 KeywordScore(word=4441, frequency=90, coefficient=2.444449775302257),
 KeywordScore(word=5880, frequency=221, coefficient=1.7612558845790212),
 KeywordScore(word=8215, frequency=105, coefficient=1.4857148956888908),
 KeywordScore(word=4343, frequency=46, coefficient=1.414853810512473),
 KeywordScore(word=2913, frequency=119, coefficient=1.3364152179720257),
 KeywordScore(word=3527, frequency=195, coefficient=1.1779128980432874),
 KeywordScore(word=3261, frequency=216, coefficient=1.1626862220365641),
 KeywordScore(word=6208, frequency=77, coefficient=1.117745487267983)]

index2word를 x와 함께 train()에 넣는다면 키워드 추출 결과는 모두 word index가 str로 바뀌어서 입력됩니다. 만약 word index가 index2word의 boundary를 넘어가는 경우라면 Unk101과 같이 Unk%d의 형식으로 출력됩니다. 

In [4]:
from soykeyword.lasso import LassoKeywordExtractor

lassobased_extractor = LassoKeywordExtractor(
    costs=[500, 200, 100, 50, 10, 5, 1, 0.1],
    min_tf=20, 
    min_df=10
)

lassobased_extractor.train(x, _idx_to_vocab)

index2word가 train에 입력되면 '아이오아이'가 포함된 문서의 index를 가져올 수도 있습니다. 

In [5]:
documents = lassobased_extractor.get_document_index('아이오아이')

extract_from_docs(documents)는 documents를 positive set으로, documents가 아닌 모든 문서를 negative set으로 선택한 뒤, 이 둘을 구분하는 L1 Logistic regression을 학습합니다. 이 때에는 '아이오아이'라는 단어가 포함된 문서와 포함되지 않는 문서를 구분하는 classifier를 학습하였기 때문에 '아이오아이'가 최상위 키워드로 선택됩니다. 하지만 이는 정답을 알고 문제를 푸는 격입니다. extract_from_docs는 이 경우보다는 본인이 positive set으로 선택하고 싶은 문서 집합이 명확히 있을 때 이용하는 방법입니다.  

In [6]:
keywords = lassobased_extractor.extract_from_docs(documents, min_num_of_keywords=30)
keywords

4 keywords extracted from 0.100 cost
4 keywords extracted from 1.000 cost
4 keywords extracted from 5.000 cost
5 keywords extracted from 10.000 cost
13 keywords extracted from 50.000 cost
20 keywords extracted from 100.000 cost
26 keywords extracted from 200.000 cost
51 keywords extracted from 500.000 cost


[KeywordScore(word='아이오아이', frequency=270, coefficient=16.882347051884235),
 KeywordScore(word='산들', frequency=90, coefficient=1.3354261278067767),
 KeywordScore(word='선의', frequency=40, coefficient=1.2600250932912496),
 KeywordScore(word='엠카운트다운', frequency=221, coefficient=1.2288369418069456),
 KeywordScore(word='드림', frequency=119, coefficient=1.0716780419523757),
 KeywordScore(word='키미', frequency=297, coefficient=0.9496554139306275),
 KeywordScore(word='파워풀', frequency=152, coefficient=0.7491257724832632),
 KeywordScore(word='챔피언', frequency=105, coefficient=0.7378773519351083),
 KeywordScore(word='고양시', frequency=264, coefficient=0.6027859304587604),
 KeywordScore(word='뮤직', frequency=195, coefficient=0.5988402912862572),
 KeywordScore(word='컴백', frequency=536, coefficient=0.5381035198692512),
 KeywordScore(word='뉴스1스타', frequency=357, coefficient=0.4477513167871695),
 KeywordScore(word='신용재', frequency=82, coefficient=0.4049837277600391),
 KeywordScore(word='세련', frequency=192, 

extract_from_word(aspect_word)는 aspect_word가 들어간 문서 집합을 positive set으로, 그렇지 않은 문서 집합을 negative set으로 선택하여 이를 구분하는 classifier를 학습하는 것입니다. 이 때, positive set에서 aspect_word를 제외합니다. 이렇게 할 경우, aspect_word 때문에 잘못된 hyperplane이 선택되는 문제를 해결할 수 있습니다. 

2016-10-20에는 '아이오아이'가 마지막 곡인 '너무너무너무'곡으로 활동하던 시절입니다. 그렇기 때문에 '너무너무너무'와 '엠카운트다운'과 같은 단어가 키워드로 선택되었음을 볼 수 있습니다. 

In [7]:
keywords = lassobased_extractor.extract_from_word('아이오아이', min_num_of_keywords=30)
keywords

28 keywords extracted from 0.100 cost
63 keywords extracted from 1.000 cost


[KeywordScore(word='너무너무너무', frequency=86, coefficient=3.8117099150802307),
 KeywordScore(word='선의', frequency=40, coefficient=3.2619544803567555),
 KeywordScore(word='산들', frequency=90, coefficient=2.446948735843341),
 KeywordScore(word='엠카운트다운', frequency=221, coefficient=1.7610348257902289),
 KeywordScore(word='챔피언', frequency=105, coefficient=1.498461786927164),
 KeywordScore(word='사나', frequency=46, coefficient=1.4203424184189373),
 KeywordScore(word='드림', frequency=119, coefficient=1.348433253864504),
 KeywordScore(word='뮤직', frequency=195, coefficient=1.1897826640627807),
 KeywordScore(word='먹고', frequency=216, coefficient=1.1682337894307808),
 KeywordScore(word='완전체', frequency=77, coefficient=1.1134199864576417),
 KeywordScore(word='일산', frequency=194, coefficient=0.969133662540626),
 KeywordScore(word='세련', frequency=192, coefficient=0.9463765698475632),
 KeywordScore(word='같이', frequency=918, coefficient=0.8434273084655862),
 KeywordScore(word='뉴스1스타', frequency=357, coeffic

'최순실'의 경우에도 '박근혜-최순실 게이트'사건이 시작되는 단계이기 때문에 해당 뉴스들이 나오고 있었습니다. '최순실'이라는 단어가 들어간 문서를 선택하는 기준은 [게이트, 정유라, 고영태, 이화여대, 미르재단]과 같은 단어가 선택되어 해당 날짜의 뉴스를 잘 요약하고 있음을 볼 수 있습니다. 

In [8]:
keywords = lassobased_extractor.extract_from_word('최순실', min_num_of_keywords=30)
keywords

78 keywords extracted from 0.100 cost


[KeywordScore(word='게이트', frequency=303, coefficient=4.203402809567516),
 KeywordScore(word='정유라', frequency=329, coefficient=2.037140460890112),
 KeywordScore(word='고영태', frequency=65, coefficient=1.721663656344356),
 KeywordScore(word='편파기소', frequency=54, coefficient=1.2857257231065549),
 KeywordScore(word='최서원', frequency=94, coefficient=1.2762140387693015),
 KeywordScore(word='비선', frequency=288, coefficient=1.0134780099708183),
 KeywordScore(word='씨와', frequency=372, coefficient=0.7832048635945356),
 KeywordScore(word='이화여대', frequency=651, coefficient=0.7236313130906012),
 KeywordScore(word='송민순', frequency=821, coefficient=0.6901110483802853),
 KeywordScore(word='뉴스', frequency=4682, coefficient=0.5788237881384294),
 KeywordScore(word='연설문', frequency=204, coefficient=0.49729393663321186),
 KeywordScore(word='침묵', frequency=223, coefficient=0.4263484031947302),
 KeywordScore(word='박근혜', frequency=1445, coefficient=0.40730356621805713),
 KeywordScore(word='실세', frequency=309, co

## Proportion ratio for keyword extraction

proportion ratio 를 이용하는 방법도 soykeyword 에 올려두었습니다. Interface 는 Lasso 와 동일합니다. 

extract_from_word 와 extract_from_docs 를 제공합니다. 

In [9]:
from soykeyword.proportion import MatrixbasedKeywordExtractor

proportion_based_extractor = MatrixbasedKeywordExtractor(
    min_tf=20, 
    min_df=2,
    verbose=True)

proportion_based_extractor.train(x, _idx_to_vocab)

MatrixbasedKeywordExtractor trained


In [10]:
keywords = proportion_based_extractor.extract_from_word(
    '아이오아이', min_score=0.8, min_frequency=100)
keywords[:10]

[KeywordScore(word='아이오아이', frequency=270, score=1.0),
 KeywordScore(word='엠카운트다운', frequency=221, score=0.9978307775631691),
 KeywordScore(word='펜타곤', frequency=104, score=0.9934422266805437),
 KeywordScore(word='잠깐', frequency=162, score=0.9929667382454291),
 KeywordScore(word='엠넷', frequency=125, score=0.9907514986652862),
 KeywordScore(word='걸크러쉬', frequency=111, score=0.99017203825805),
 KeywordScore(word='타이틀곡', frequency=311, score=0.9869906112674688),
 KeywordScore(word='코드', frequency=105, score=0.9867835556082788),
 KeywordScore(word='본명', frequency=105, score=0.98596911773225),
 KeywordScore(word='엑스', frequency=101, score=0.9847950780631249)]